# The composition over inheritance principle 

In [1]:
# imports 
import sys
import syslog

In [9]:
# initial class
class Logger(object):
    def __init__(self, file):
        self.file = file
    
    def log(self, message):
        self.file.write(message + '\n')
        self.file.flush()

# now, we need two more classes that send message to elsewhere

class SocketLogger(Logger):
    def __init__(self, sock):
        self.sock = sock
    
    def log(self, message):
        self.sock.sendall(message + '\n').encode('ascii')

class SyslogLogger(Logger):
    def __init__(self, priority):
        self.priority = priority
    
    def log(self, message):
        syslog.syslog(self.priority, message)

# Now if we want to add filters, we can create a filtered logger creating a new subclass

class FilteredLogger(Logger):
    def __init__(self, pattern, file):
        self.pattern = pattern
        super().__init__(file)
    
    def log(self, message):
        if self.pattern in message:
            super().log(message)

# call filtered logger
f = FilteredLogger('Error', sys.stdout)
f.log('Ignored: this is not important')
f.log('Error: but you want to see this')

Error: but you want to see this


#### The problem
Now if we need to filter and write then to socket instead of file
we cannot, we will need to subclass and create another class with ex: "FilteredSocketLogger"

Maybe the programmer will get lucky and no further combinations will be needed. But in the general case the application will wind up with 3×2=6 classes:

Logger            FilteredLogger
SocketLogger      FilteredSocketLogger
SyslogLogger      FilteredSyslogLogger

The total number of classes will increase geometrically if m and n both continue to grow. 
This is the “proliferation of classes” and “explosion of subclasses” that the Gang of Four want to avoid.

But we can solve this problem.

# Solution 1: The Adapter Pattern

One solution is the Adapter Pattern: to decide that the original logger class doesn’t need to be improved, 
because any mechanism for outputting messages can be wrapped up to look like the file object that the logger is expecting.

* So we keep the original Logger.
* And we also keep the FilteredLogger.
* But instead of creating destination-specific subclasses, 
we adapt each destination to the behavior of a file and then pass the adapter to a Logger as its output file.

In [18]:
import socket

class FileLikeSocket:
    def __init__(self, sock):
        self.sock = sock
    
    def write(self, message):
        self.sock.sendall(message.encode('ascii'))
    
    def flush(self):
        pass

class FileLikeSyslog:
    def __init__(self, priority):
        self.priority = priority
    
    def write(self, message):
        syslog.syslog(self.priority, message)
    
    def flush(self):
        pass

# create 
sock1, sock2 = socket.socketpair()

fs = FileLikeSocket(sock1)
logger = FilteredLogger(file=fs, pattern='Error')
logger.log('Warning: message number one')
logger.log('Error: message number two')

print('The socket received: %r' % sock2.recv(512))

The socket received: b'Error: message number two\n'


We can see that using adapter patter we can avoid subclass explosion. 
Logger object and adapter object can easily mixed and matched at the runtime without need to create any further subclass.